##Installing the required packages

In [ ]:
#!pip install pycparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install anytree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 642 kB/s 


In [ ]:
#!pip show pycparser

Name: pycparser
Version: 2.21
Summary: C parser in Python
Home-page: https://github.com/eliben/pycparser
Author: Eli Bendersky
Author-email: eliben@gmail.com
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: cffi


In [ ]:
#import os
#os.listdir('/usr/local/lib/python3.7/dist-packages/pycparser')

['c_parser.py',
 '__pycache__',
 'plyparser.py',
 'ast_transforms.py',
 'lextab.py',
 '_c_ast.cfg',
 '__init__.py',
 'c_ast.py',
 '_ast_gen.py',
 'utils',
 'ply',
 '_build_tables.py',
 'c_generator.py',
 'c_lexer.py',
 'yacctab.py']

In [ ]:
'''
#to remove comments 
import re
def comment_remover(text):
    def replacer(match):
        s = match.group(0)
        if s.startswith('/'):
            return ""
        else:
            return s
    pattern = re.compile(
        r'//.*?$|/\*.*?\*/|\'(?:\\.|[^\\\'])*\'|"(?:\\.|[^\\"])*"',
        re.DOTALL | re.MULTILINE
    )
    return re.sub(pattern, replacer, text)
'''

In [ ]:
'''
import pycparser
filename = '/content/testPP.c'
ast = pycparser.parse_file(filename, use_cpp=True)        
ast.show()
'''

"\nimport pycparser\nfilename = '/content/testPP.c'\nast = pycparser.parse_file(filename, use_cpp=True)        \nast.show()\n"

In [ ]:
'''
for i in ast.ext[2]:
  print("--------------")
  print(i.children())
'''

'\nfor i in ast.ext[2]:\n  print("--------------")\n  print(i.children())\n'

##Generating AST for the code

In [ ]:
import pycparser
from pycparser import c_parser

def get_ast(filename):
    #import pycparser
    ast = pycparser.parse_file(filename, use_cpp=True)  
    #parser = c_parser.CParser()
    #ast = parser.parse(func, filename='<none>')
    #print(ast)
    return ast

##Getting the tokens from the nodes

In [ ]:
def get_token(node):
    token = ''
    if isinstance(node, pycparser.c_ast.FileAST):
        token = node.__class__.__name__
        return token
    elif isinstance(node[1], str):
        token = node[1]
        return token
        #print(token)
    elif isinstance(node[1], pycparser.c_ast.Node):
        token = node[1].__class__.__name__
    #print(token)
    
    if len(get_children(node))==0:
        attr_names = node[1].attr_names
        if attr_names:
            if 'names' in attr_names:
                token = node[1].names[0]
            elif 'name' in attr_names:
                token = node[1].name
            else:
                token = node[1].value
    else:
        if token == 'TypeDecl':
            token = node[1].declname
        if node[1].attr_names:
            attr_names = node[1].attr_names
            if 'op' in attr_names:
                if node[1].op[0] == 'p':
                    token = node[1].op[1:]
                else:
                    token = node[1].op
 
    if token == '':
        token = node[1].__class__.__name__
    return token

##Getting the children nodes and getting a tree 

In [ ]:
from anytree import Node

def get_children(root):
    if isinstance(root, pycparser.c_ast.FileAST):
        children = root.children()
    elif isinstance(root[1], pycparser.c_ast.Node):
        children = root[1].children()
    elif isinstance(root, set):
        children = list(root)
    else:
        children = []

    def expand(nested_list):
        for item in nested_list:
            if isinstance(item, list):
                for sub_item in expand(item):
                    yield sub_item
            elif item:
                yield item

    return list(expand(children))

def get_trees(current_node, parent_node, order,nodes):
    token, children = get_token(current_node), get_children(current_node)
    node = Node([order,token], parent=parent_node, order=order)
    #print(parent_node)
    #if(parent_node.name=='')
    p = str(parent_node.name[0])+"--"+str(parent_node.name[1])
    c = str(order)+"--"+str(token)
    nodes.append((p,c))
    #print(nodes)
    for child_order in range(len(children)):
        get_trees(children[child_order], node, order+str(random.randint(1,100000)+1),nodes)

In [ ]:
#c_code = ast
#ast = program_parser(filename)

In [ ]:
from anytree import Node
def get_c_nodes(filename):
  #get the ast
  c_ast = get_ast(filename)
  
  # Initialize head node of the code.
  head = Node(["1",get_token(c_ast)])
  
  # Recursively construct AST tree and get all the nodes in nodes list
  nodes = []
  #nodes.append((None,str(head.name[1])+str(head.name[0])))
  for child_order in range(len(get_children(c_ast))):
      get_trees(get_children(c_ast)[child_order], head, "1"+str(int(child_order)+1),nodes)
  
  return nodes

##Calling the function to get the nodes

In [ ]:
filename = '/content/testPP.c'
nodes = get_c_nodes(filename)

In [ ]:
for i in nodes:
  print(i)

('1--FileAST', '11--Typedef')
('11--Typedef', '111--size_t')
('111--size_t', '1111--int')
('1--FileAST', '12--Typedef')
('12--Typedef', '121--__builtin_va_list')
('121--__builtin_va_list', '1211--int')
('1--FileAST', '13--Typedef')
('13--Typedef', '131--__gnuc_va_list')
('131--__gnuc_va_list', '1311--int')
('1--FileAST', '14--Typedef')
('14--Typedef', '141--va_list')
('141--va_list', '1411--int')
('1--FileAST', '15--Typedef')
('15--Typedef', '151--__int8_t')
('151--__int8_t', '1511--int')
('1--FileAST', '16--Typedef')
('16--Typedef', '161--__uint8_t')
('161--__uint8_t', '1611--int')
('1--FileAST', '17--Typedef')
('17--Typedef', '171--__int16_t')
('171--__int16_t', '1711--int')
('1--FileAST', '18--Typedef')
('18--Typedef', '181--__uint16_t')
('181--__uint16_t', '1811--int')
('1--FileAST', '19--Typedef')
('19--Typedef', '191--__int_least16_t')
('191--__int_least16_t', '1911--int')
('1--FileAST', '110--Typedef')
('110--Typedef', '1101--__uint_least16_t')
('1101--__uint_least16_t', '11011-

In [ ]:
#from anytree import RenderTree
#print(RenderTree(head))

In [ ]:
'''
from pprint import pprint  # just for nice printing
#from anytree import AnyNode
from anytree.exporter import DictExporter
exporter = DictExporter()
pprint(exporter.export(head))
'''

'\nfrom pprint import pprint  # just for nice printing\n#from anytree import AnyNode\nfrom anytree.exporter import DictExporter\nexporter = DictExporter()\npprint(exporter.export(head))\n'

In [ ]:
'''
#from _typeshed import OpenBinaryModeUpdating
from pycparser import c_ast, parse_file
import pycparser
class Visitor(c_ast.NodeVisitor):
  
    def __init__(self,current_parent,op):
        self.current_parent = current_parent
        self.op = op
        pass
    
    #def visit(self, node:c_ast.FileAST):
    #    #print(node)
    #    self.generic_visit(node);
   
    
    
    def visit(self, node):
        """ Visit a node.
        """

        if self._method_cache is None:
            self._method_cache = {}

        visitor = self._method_cache.get(node.__class__.__name__, None)
        if visitor is None:
            method = 'visit_' + node.__class__.__name__
            visitor = getattr(self, method, self.generic_visit)
            self._method_cache[node.__class__.__name__] = visitor

        return visitor(node)
        

    def generic_visit(self, node):
        """ Called if no explicit visitor function exists for a
            node. Implements preorder visiting of the node.
        """
        oldparent = self.current_parent
        self.current_parent = node
        op.append((get_token(oldparent),get_token(node)))
        for c in node:
            self.visit(c)
        self.current_parent = oldparent

'''

In [ ]:
#nodes

##Removing typedef nodes and their children

In [ ]:
#getting the parent and children in separate lists

def get_parent_child(nodes):
  parents=[]
  children=[]

  for i in nodes:
    parents.append(i[0])
    children.append(i[1])

  return(parents,children)

In [ ]:
#getting all the children of the given node

def get_all_children(parents,node,children,rem_indices):
  for i in range(len(parents)):
    if node==parents[i]: #check if the node is in the list of parents 
      #print(node)
      #print(i)
      rem_indices.append(i) #if yes append the index 

In [ ]:
#removing all the typedef nodes and the children

def get_typedef(parents,children):
  rem_indices=[] #to get the indices to be removed
  i=0 #to get the current index

  #traverse the whole list of parents
  while(i<len(parents)):
    
    if 'Typedef' in children[i]:
      rem_indices.append(i) #append i to rem_indices
      child=children[i] #get the child of the current parent
      
      while((child in parents)):
        get_all_children(parents,child,children,rem_indices) #append all the children of the current parent
        i+=1 
        child=children[i] #get the children of the current parent
      
    i+=1 #increment the index
    
  return rem_indices
      

In [ ]:
#print(rem_indices)

In [ ]:
#for i in rem_indices:
#  print(nodes[i])

In [ ]:
len(nodes)

722

In [ ]:
#traverse through the rem_indices list and remove the nodes 

def remove_nodes(rem_indices,nodes):
  rem_nodes=[]
  
  for i in rem_indices:
    rem_nodes.append(nodes[i])
  
  nodes_1=list(set(nodes)-set(rem_nodes))
  
  return nodes_1

In [ ]:
#function call to get the req codes

def get_req_nodes(nodes):
  l= get_parent_child(nodes)
  parents,children=l[0],l[1]
  
  rem_indices=get_typedef(parents,children)
  
  nodes_1 = remove_nodes(rem_indices,nodes)
  
  return nodes_1

In [ ]:
nodes = get_req_nodes(nodes)

[('1194111--ParamList', '11941111--Typename'), ('119423--Decl', '1194232--10'), ('1194222--/', '11942222--sizeof'), ('119321232123---', '1193212321232--1'), ('11942422--ExprList', '119424223---'), ('1193212412--ExprList', '11932124121--arr'), ('1193212--Compound', '11932121--Decl'), ('11942422--ExprList', '119424221--arr'), ('1193212412--ExprList', '11932124124--x'), ('119425--TernaryOp', '1194251--=='), ('11932123212--ExprList', '119321232121--arr'), ('11931--Decl', '119311--FuncDecl'), ('1193111--ParamList', '11931111--Decl'), ('1193111--ParamList', '11931113--Decl'), ('11942111--arr', '119421111--int'), ('11942--Compound', '119422--Decl'), ('119321231-->', '1193212312--x'), ('1194212--InitList', '11942121--2'), ('1193111111--arr', '11931111111--int'), ('1193212211--ArrayRef', '11932122111--arr'), ('119426--Return', '1194261--0'), ('1194241--result', '11942411--int'), ('119322--Return', '1193221---'), ('11932123--If', '119321232--Return'), ('119424--Decl', '1194242--FuncCall'), ('119

In [ ]:
len(nodes)

134

##Writing nodes into a file

In [ ]:
with open('c_nodes.txt', 'w') as fp:
     for item in nodes:
        # write each item on a new line
        #print(item[0],item[1])
        item0=rem_newline(item[0])
        item1=rem_newline(item[1])
        print(item0,item1)
        fp.write(str(item0)+","+str(item1)+"\n")
    print('Done')

Done
